In [1]:
import argparse
import os
import sys
import shutil

import cv2
from lxml import etree, objectify

# 将标签信息写入xml
from tqdm import tqdm

In [64]:
def save_anno_to_xml(filename, size, objs, save_path):
    E = objectify.ElementMaker(annotate=False)
    anno_tree = E.annotation(
        E.folder("DATA"),
        E.filename(filename),
        E.source(
            E.database("The VOC Database"),
            E.annotation("PASCAL VOC"),
            E.image("flickr")
        ),
        E.size(
            E.width(size[1]),
            E.height(size[0]),
            E.depth(size[2])
        ),
        E.segmented(0)
    )
    for obj in objs:
        E2 = objectify.ElementMaker(annotate=False)
        if obj[0] == 'nopolyp':
            anno_tree2 = E2.object(
            E.name(obj[0]),
            E.positive(0),
            E.truncated(0),
            E.difficult(0),
            E.bndbox(
                E.xmin(obj[1][0]),
                E.ymin(obj[1][1]),
                E.xmax(obj[1][2]),
                E.ymax(obj[1][3])
                )
            )
        else:
            anno_tree2 = E2.object(
            E.name(obj[0]),
            E.positive(1),
            E.truncated(0),
            E.difficult(0),
            E.bndbox(
                E.xmin(obj[1][0]),
                E.ymin(obj[1][1]),
                E.xmax(obj[1][2]),
                E.ymax(obj[1][3])
                )
            )
        anno_tree.append(anno_tree2)

    anno_path = os.path.join(save_path, filename[:-3] + "xml")
    etree.ElementTree(anno_tree).write(anno_path, pretty_print=True)
    


def xywhn2xyxy(bbox, size):
    bbox = list(map(float, bbox))
    size = list(map(float, size))
    xmin = (bbox[0] - bbox[2] / 2.) * size[1]
    ymin = (bbox[1] - bbox[3] / 2.) * size[0]
    xmax = (bbox[0] + bbox[2] / 2.) * size[1]
    ymax = (bbox[1] + bbox[3] / 2.) * size[0]
    box = [xmin, ymin, xmax, ymax]
    return list(map(int, box))


def parseXmlFilse(image_path, anno_path,class_path, save_path):
    global images_nums, category_nums, bbox_nums
    assert os.path.exists(image_path), "ERROR {} dose not exists".format(image_path)
    assert os.path.exists(anno_path), "ERROR {} dose not exists".format(anno_path)
    if os.path.exists(save_path):
        shutil.rmtree(save_path)
    os.makedirs(save_path)

    category_set = []
    with open(class_path, 'r') as f:
        for i in f.readlines():
            category_set.append(i.strip())
    category_nums = len(category_set)
    category_id = dict((k, v) for k, v in enumerate(category_set))

    images = [os.path.join(image_path, i) for i in os.listdir(image_path)]
    files = [os.path.join(anno_path, i) for i in os.listdir(anno_path)]
    images_index = dict((v.split(os.sep)[-1][:-4], k) for k, v in enumerate(images))
    images_nums = len(images)

    for file in tqdm(files, ncols = 80):
        if os.path.splitext(file)[-1] != '.txt' or 'classes' in file.split(os.sep)[-1]:
            continue
        if file.split(os.sep)[-1][:-4] in images_index:
            index = images_index[file.split(os.sep)[-1][:-4]]
            img = cv2.imread(images[index])
            shape = img.shape
            filename = images[index].split(os.sep)[-1]
        else:
            continue
        objects = []
        if os.path.getsize(file) == 0:
            objects.append(['nopolyp',[0,0,0,0]])
            continue
        with open(file, 'r') as fid:
            ## LDPolypVideo
            # for line in fid.readlines()[1:]:
            #     xmin, ymin, xmax, ymax     = line.strip().split(' ')
            #     xmin, ymin, xmax, ymax     = int(xmin), int(ymin), int(xmax), int(ymax)
            #     obj = [category_id[0],[xmin,ymin,xmax,ymax]]    
            #     objects.append(obj)    
            ## yolo
            for i in fid.readlines():
                i = i.strip().split() 
                category = int(i[0])
                category_name = category_id[category]
                bbox = xywhn2xyxy((i[1], i[2], i[3], i[4]), shape)
                obj = [category_name, bbox]
                objects.append(obj)
        bbox_nums += len(objects)
        save_anno_to_xml(filename, shape, objects, save_path)
 

In [65]:
"""
脚本说明：
    本脚本用于将yolo/LDPolypVideo格式的标注文件.txt转换为voc格式的标注文件.xml
参数说明：
    anno_path:标注文件txt存储路径
    save_path:xml文件输出的文件夹
    image_path:图片路径
"""
images_nums = 0
category_nums = 0
bbox_nums = 0
anno_path = '../../../Dataset/Five_TestDataset/CVC-300/labels'
save_path = '../../Polyp-Video-Detection/STFT/datasets/CVC-300'
image_path = '../../../Dataset/Five_TestDataset/CVC-300/images'
class_path = '/data1/YiZhenyu/Project/polyp-segmentation/Polyp-PVT/dataset/TrainDataset/CVC550_Kvasir900/classes.txt'


parseXmlFilse(image_path, anno_path,class_path, save_path)
print("image nums: {}".format(images_nums))
print("category nums: {}".format(category_nums))
print("bbox nums: {}".format(bbox_nums))

100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 158.27it/s]

image nums: 60
category nums: 1
bbox nums: 60


In [44]:
file = '1.txt'
with open(file, 'r') as fid:
    if not fid.read().strip():
        print(1)


1


In [3]:
# generate LDPolypVideo dataset ImageSets
import os
lens = 0
train_paths = []
test_paths = []
path = '/memory/yizhenyu/dataset/LDPolypVideo/Annotations'
for file in os.listdir(path):
    # if int(file) > 100: continue
    file_path = os.path.join(path,file)
    folder_len = len(os.listdir(file_path))
    for name in os.listdir(file_path):
        path0 = os.path.join(file_path,name)
        with open(path0,'r') as f:
            if f.read().strip() == '0':
                continue
            elif int(file) <= 100: train_paths.append([os.path.join(file,name),folder_len])
            else: test_paths.append([os.path.join(file,name),folder_len])

In [1]:
# for path in train_paths:
#     print(path)
#     break
len(train_paths)

NameError: name 'train_paths' is not defined

In [7]:
save_path = '/memory/yizhenyu/dataset/LDPolypVideo/ImageSets/LDPolypVideo_val_videos.txt'
with open(save_path,'a') as f:
    for path in test_paths:
        f.write(path[0][:-4]+' '+str(path[1]))
        f.write('\n')

In [31]:
for file in os.listdir(path):
    if 99 <= int(file) <= 101 :
        print(file == '101')

True
False
False


In [13]:
path = '/memory/yizhenyu/dataset/LDPolypVideo/Annotations/100/0001.txt'

In [5]:
with open(path, 'r') as f:
    lines = f.readlines()
    for line in lines[1:]:
        xmin, ymin, xmax, ymax     = line.strip().split(' ')
        xmin, ymin, xmax, ymax     = int(xmin), int(ymin), int(xmax), int(ymax)
        mask1[ymin:ymax, xmin:xmax] = 1

'0\n'

In [20]:
cont.strip() == '0'

True

In [18]:
x = 342
# len(str(x))
for _ in range(4-len(str(x))):
    x = '0'+ str(x)

In [22]:
x

'0342'

In [2]:
class ss:
    def pp(self):
        print(1)

    def qq(self):
        self.pp()

In [5]:
a = ss()
a.qq()

1
